In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='NDX', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2018,4,1), train_end=datetime(2025,4,1),
        test_start=datetime(2023,4,1),  test_end=datetime(2024,4,1),
        label_method='random', search_type='clusters', search_subtype='kmeans',
        n_models=1, n_jobs=8, debug=False,
    ),
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2018,4,1), train_end=datetime(2025,4,1),
        test_start=datetime(2023,4,1),  test_end=datetime(2024,4,1),
        label_method='random', search_type='clusters', search_subtype='kmeans',
        n_models=1, n_jobs=8, debug=False,
    ),
    dict(
        symbol='GDAXI', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2018,4,1), train_end=datetime(2025,4,1),
        test_start=datetime(2023,4,1),  test_end=datetime(2024,4,1),
        label_method='random', search_type='clusters', search_subtype='kmeans',
        n_models=1, n_jobs=8, debug=False,
    ),
]

# Crear tag para cada configuración
for cfg in configs:
    # Construir el tag asegurando que no haya dobles guiones bajos por campos vacíos
    tag_parts = [
        cfg['symbol'],
        cfg['timeframe'],
        cfg['direction'],
        cfg['label_method'][:2],
        cfg['search_type'][:3],
        (cfg.get('search_subtype') or '')[:2],
    ]
    # Filtrar partes vacías y unir con "_"
    cfg["tag"] = "_".join([part for part in tag_parts if part]).strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 5000
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[NDX_H1_buy_ra_clu_km] modelo 0 trial 55/5044 score=0.407741 avg=0.78s mem=400.00MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 44/5044 score=0.377703 avg=1.24s mem=763.06MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 45/5044 score=0.377703 avg=2.81s mem=796.09MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 46/5044 score=0.377703 avg=3.31s mem=846.07MB 
[NDX_H1_buy_ra_clu_km] modelo 0 trial 56/5044 score=0.407741 avg=3.20s mem=1384.69MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 47/5044 score=0.377703 avg=4.33s mem=981.60MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 48/5044 score=0.377703 avg=4.27s mem=980.86MB 
[GDAXI_H1_buy_ra_clu_km] modelo 0 trial 45/5044 score=0.362846 avg=5.18s mem=1705.52MB 
[NDX_H1_buy_ra_clu_km] modelo 0 trial 57/5044 score=0.407741 avg=4.29s mem=1410.20MB 
[GDAXI_H1_buy_ra_clu_km] modelo 0 trial 46/5044 score=0.362846 avg=5.71s mem=1693.65MB 
[NDX_H1_buy_ra_clu_km] modelo 0 trial 58/5044 score=0.407741 avg=5.65s mem=1468.26MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 